模型融合训练，融合InceptionV3、Xception、InceptionResNetV2

In [21]:
import numpy as np
np.random.seed(19906)
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from keras.preprocessing import image
import cv2
import os
import h5py
%matplotlib inline
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import EarlyStopping
from keras.models import *
from keras.layers import *
from keras.layers.core import Dropout
from keras.optimizers import Adam, SGD, RMSprop
from keras.regularizers import l2
from keras.applications.inception_v3 import InceptionV3, preprocess_input as inceptionv3_preinput
from keras.applications.xception import Xception, preprocess_input as xception_preinput
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input as inception_resnet_preinput

In [18]:
train_dir = "data/imgs/train2"
val_dir = "data/imgs/val2"
test_dir = "data/imgs/test1"
saved_weights_dir = "saved_weights"
batch_size = 128

In [19]:
def write_gap(MODEL, weight_file, image_size, preprocess_fun=None):
    input_tensor = Input((*image_size, 3))
    base_model = MODEL(input_tensor=input_tensor, weights=None, include_top=False)
    
    model = Model(inputs=base_model.input, outputs=GlobalAveragePooling2D()(base_model.output), name=base_model.name)
    weights_path = os.path.join(saved_weights_dir, weight_file)
    model.load_weights(weights_path, by_name=True)
    print("loaded model weights: ", model.name, ", weights path:", weights_path)
    
    # 训练集图像生成器
    train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_fun,
        rotation_range=10.,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.1,
        zoom_range=0.1,
        rescale=1./255
    )

    # 验证集图像生成器
    val_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_fun,
        rescale=1./255)
    
    test_datagen = ImageDataGenerator(preprocessing_function=preprocess_fun, rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=image_size,
        batch_size=batch_size,
        shuffle=False,
        class_mode='categorical')

    val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=image_size,
        batch_size=batch_size,
        shuffle=False,
        class_mode='categorical')
    
    test_generator = test_datagen.flow_from_directory(test_dir, image_size, shuffle=False, 
                                             batch_size=batch_size, class_mode=None)
    
    train = model.predict_generator(train_generator, verbose=1)
    valid = model.predict_generator(val_generator, verbose=1)
    
    print("begin create bottleneck file:")
    with h5py.File(os.path.join(saved_weights_dir, "bottleneck_%s.h5") % model.name) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("valid", data=valid)
        h.create_dataset("label", data=train_generator.classes)
        h.create_dataset("valid_label", data=valid_generator.classes)
    print("create bottleneck file done for model: ", model.name)
    
    test = model.predict_generator(test_generator, verbose=1)
    
    print("begin create test bottleneck file:")
    with h5py.File(os.path.join(saved_weights_dir, "bottleneck_test_%s.h5") % model.name) as h:
        h.create_dataset("test", data=test)
    print("create test bottleneck file done for model: ", model.name)

In [ ]:
write_gap(InceptionV3, "inception_v3_model.h5", (299, 299), inceptionv3_preinput)
write_gap(Xception, "xception_model.h5", (299, 299), xception_preinput)
write_gap(InceptionResNetV2, "inception_resnet_v2_model.h5", (299, 299), inception_resnet_preinput)

loaded model weights:  inception_v3 , weights path: saved_weights/inception_v3_model.h5
Found 41850 images belonging to 10 classes.
Found 1399 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.
268/327 [=======================>......] - ETA: 5:12